In [33]:
import pandas as pd
from sentence_transformers import SentenceTransformer

In [34]:
!pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
df=pd.read_csv("../archive/titles.csv")

In [4]:
df.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,tm1300,A Charlie Brown Christmas,MOVIE,When Charlie Brown complains about the overwhe...,1965,G,25,"['comedy', 'drama', 'family', 'music', 'animat...",['US'],NaN,tt0059026,8.3,40328.0,10.848,7.688
1,tm71152,"It's the Great Pumpkin, Charlie Brown",MOVIE,"This classic ""Peanuts"" tale focuses on the thu...",1966,G,25,"['animation', 'comedy', 'family']",['US'],NaN,tt0060550,8.2,20299.0,12.458,7.474
2,tm2562,A Charlie Brown Thanksgiving,MOVIE,"Turkey, cranberries, pumpkin pie... and the Pe...",1973,G,25,"['family', 'animation', 'comedy']",['US'],NaN,tt0068359,7.7,11406.0,12.591,7.311
3,tm125274,"Be My Valentine, Charlie Brown",MOVIE,It's Valentine's Day again and Charlie Brown d...,1975,G,25,"['animation', 'family', 'romance', 'comedy']",['US'],NaN,tt0072687,7.4,2777.0,6.981,7.000
4,tm106607,"It's the Easter Beagle, Charlie Brown",MOVIE,"Charlie Brown, Linus and the entire Peanuts ga...",1974,G,25,"['family', 'animation', 'comedy']",['US'],NaN,tt0071679,7.5,3001.0,6.172,6.600


In [5]:
df.columns

Index(['id', 'title', 'type', 'description', 'release_year',
       'age_certification', 'runtime', 'genres', 'production_countries',
       'seasons', 'imdb_id', 'imdb_score', 'imdb_votes', 'tmdb_popularity',
       'tmdb_score'],
      dtype='object')

In [6]:
df["seasons"]=df["seasons"].fillna("0")

In [7]:
df.isna().sum()

id                       0
title                    0
type                     0
description              0
release_year             0
age_certification       11
runtime                  0
genres                   0
production_countries     0
seasons                  0
imdb_id                  1
imdb_score               3
imdb_votes               4
tmdb_popularity          0
tmdb_score               3
dtype: int64

In [8]:
#cherchons le mode
mode_value = df['age_certification'].mode()[0]
print(mode_value)





TV-MA


In [9]:
df["age_certification"]=df["age_certification"].fillna(mode_value)

In [10]:
mode_imdbScore = df['imdb_score'].mode()[0]
print(mode_imdbScore)

7.7


In [11]:
df["imdb_score"]=df["imdb_score"].fillna(mode_imdbScore)

In [12]:
mode_tmdbScore = df['tmdb_score'].mode()[0]
print(mode_tmdbScore)

7.0


In [13]:
df["tmdb_score"]=df["tmdb_score"].fillna(mode_tmdbScore)

In [14]:
df.columns

Index(['id', 'title', 'type', 'description', 'release_year',
       'age_certification', 'runtime', 'genres', 'production_countries',
       'seasons', 'imdb_id', 'imdb_score', 'imdb_votes', 'tmdb_popularity',
       'tmdb_score'],
      dtype='object')

In [15]:
data=df[["release_year",'runtime', 'genres','seasons','production_countries','age_certification','imdb_score','tmdb_score']]

In [16]:
data.head()

,release_year,runtime,genres,seasons,production_countries,age_certification,imdb_score,tmdb_score
0,1965,25,"['comedy', 'drama', 'family', 'music', 'animat...",0,['US'],G,8.3,7.688
1,1966,25,"['animation', 'comedy', 'family']",0,['US'],G,8.2,7.474
2,1973,25,"['family', 'animation', 'comedy']",0,['US'],G,7.7,7.311
3,1975,25,"['animation', 'family', 'romance', 'comedy']",0,['US'],G,7.4,7.000
4,1974,25,"['family', 'animation', 'comedy']",0,['US'],G,7.5,6.600


In [17]:
data.isna().sum()

release_year            0
runtime                 0
genres                  0
seasons                 0
production_countries    0
age_certification       0
imdb_score              0
tmdb_score              0
dtype: int64

In [18]:
data_two=data

In [19]:
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")


In [30]:
data.isna().sum()

release_year            0
runtime                 0
genres                  0
seasons                 0
production_countries    0
age_certification       0
imdb_score              0
tmdb_score              0
dtype: int64

In [31]:
data['text'] = data['genres'] + ' ' + data['production_countries'] + ' ' + data['age_certification']

/var/folders/hd/7x_lk_y95t96lcf50ts7cbph0000gn/T/ipykernel_88871/2482216642.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['genres'] + ' ' + data['production_countries'] + ' ' + data['age_certification']


In [35]:
model = SentenceTransformer('all-MiniLM-L6-v2')

# embeddings → matrice (n_lignes, 384)
embeddings = model.encode(data['text'].tolist(), show_progress_bar=True)

Batches: 100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.22it/s]


RuntimeError: Numpy is not available

In [ ]:
# Sélection des colonnes numériques
numeric_data = data[['release_year', 'runtime', 'seasons', 'imdb_score', 'tmdb_score']].values

# Concatenation des features numériques + embeddings
X = np.hstack([numeric_data, embeddings])

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=0)
kmeans.fit(X)

# Attribuer un cluster à chaque ligne
df['cluster'] = kmeans.labels_